# Advanced Parquet Analysis Queries

This notebook demonstrates advanced queries for analyzing the Parquet files using DuckDB.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization settings
plt.style.use('bmh')
sns.set_theme()
pd.set_option('display.max_columns', None)

# Initialize DuckDB with optimized settings
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute("SET memory_limit='28GB'")
conn.execute("PRAGMA threads=8")

print("✅ DuckDB configured for analysis")

## 1. Source Record Analysis

Analyzing the distribution of source record types:

In [ ]:
source_query = """
WITH source_types AS (
 SELECT 
 json_extract_string(source.value, '$.source_type') as source_type
 FROM read_parquet('/data/parquet/*.parquet'),
 UNNEST(json_extract_array(json, '$._source.source_records')) AS source
)
SELECT 
 source_type,
 COUNT(*) as count,
 ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM source_types
GROUP BY source_type
ORDER BY count DESC;
"""

df_sources = conn.execute(source_query).fetchdf()

plt.figure(figsize=(12, 6))
plt.pie(df_sources['percentage'], labels=df_sources['source_type'], autopct='%1.1f%%')
plt.title('Distribution of Source Record Types')
plt.axis('equal')
plt.show()

## 2. Cover URL Analysis

Analyzing cover URL patterns and sources:

In [ ]:
cover_query = """
WITH cover_sources AS (
 SELECT 
 CASE
 WHEN json_extract_string(json, '$._source.file_unified_data.cover_url_best') LIKE '%archive.org%' THEN 'Internet Archive'
 WHEN json_extract_string(json, '$._source.file_unified_data.cover_url_best') LIKE '%openlibrary.org%' THEN 'Open Library'
 WHEN json_extract_string(json, '$._source.file_unified_data.cover_url_best') LIKE '%isbndb.com%' THEN 'ISBNdb'
 ELSE 'Other'
 END as cover_source,
 COUNT(*) as count
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.cover_url_best') IS NOT NULL
 GROUP BY cover_source
)
SELECT * FROM cover_sources
ORDER BY count DESC;
"""

df_covers = conn.execute(cover_query).fetchdf()
display(df_covers)

## 3. Added Date Analysis

Analyzing when records were added across different sources:

In [ ]:
dates_query = """
WITH dates AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.added_date_unified.date_ia_source') as ia_date,
 json_extract_string(json, '$._source.file_unified_data.added_date_unified.date_ol_source') as ol_date,
 json_extract_string(json, '$._source.file_unified_data.added_date_unified.date_isbndb_scrape') as isbndb_date
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.added_date_unified') IS NOT NULL
)
SELECT 
 'Internet Archive' as source,
 COUNT(*) as total_records,
 MIN(ia_date) as earliest_date,
 MAX(ia_date) as latest_date
FROM dates WHERE ia_date IS NOT NULL
UNION ALL
SELECT 
 'Open Library' as source,
 COUNT(*) as total_records,
 MIN(ol_date) as earliest_date,
 MAX(ol_date) as latest_date
FROM dates WHERE ol_date IS NOT NULL
UNION ALL
SELECT 
 'ISBNdb' as source,
 COUNT(*) as total_records,
 MIN(isbndb_date) as earliest_date,
 MAX(isbndb_date) as latest_date
FROM dates WHERE isbndb_date IS NOT NULL;
"""

df_dates = conn.execute(dates_query).fetchdf()
display(df_dates)

## 4. Language and Content Type Analysis

Analyzing language distribution and content types:

In [ ]:
lang_content_query = """
WITH stats AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type,
 UNNEST(json_extract_array(json, '$._source.file_unified_data.language_codes')) as language
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.content_type_best') IS NOT NULL
)
SELECT 
 content_type,
 language,
 COUNT(*) as count
FROM stats
GROUP BY content_type, language
HAVING count > 1000
ORDER BY count DESC;
"""

df_lang_content = conn.execute(lang_content_query).fetchdf()

# Create a pivot table
pivot_table = df_lang_content.pivot(index='content_type', columns='language', values='count').fillna(0)

plt.figure(figsize=(15, 8))
sns.heatmap(pivot_table, annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Content Type vs Language Distribution')
plt.show()

## 5. Identifier Analysis

Analyzing the distribution of different identifiers:

In [ ]:
identifier_query = """
SELECT
 'ISBN-13' as id_type,
 COUNT(DISTINCT json_extract_string(json, '$._source.file_unified_data.identifiers_unified.isbn13[0]')) as unique_count
FROM read_parquet('/data/parquet/*.parquet')
UNION ALL
SELECT
 'OCLC' as id_type,
 COUNT(DISTINCT json_extract_string(json, '$._source.file_unified_data.identifiers_unified.oclc[0]')) as unique_count
FROM read_parquet('/data/parquet/*.parquet')
UNION ALL
SELECT
 'Open Library' as id_type,
 COUNT(DISTINCT json_extract_string(json, '$._source.file_unified_data.identifiers_unified.ol[0]')) as unique_count
FROM read_parquet('/data/parquet/*.parquet');
"""

df_identifiers = conn.execute(identifier_query).fetchdf()
display(df_identifiers)

## 6. File Size Distribution by Content Type

Analyzing file sizes across different content types:

In [ ]:
filesize_query = """
WITH size_stats AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type,
 TRY_CAST(json_extract_string(json, '$._source.file_unified_data.filesize_best') AS FLOAT) / (1024*1024) as size_mb
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.filesize_best') IS NOT NULL
)
SELECT
 content_type,
 COUNT(*) as count,
 ROUND(AVG(size_mb), 2) as avg_size_mb,
 ROUND(MIN(size_mb), 2) as min_size_mb,
 ROUND(MAX(size_mb), 2) as max_size_mb
FROM size_stats
GROUP BY content_type
HAVING count > 1000
ORDER BY avg_size_mb DESC;
"""

df_filesizes = conn.execute(filesize_query).fetchdf()

plt.figure(figsize=(12, 6))
plt.bar(df_filesizes['content_type'], df_filesizes['avg_size_mb'])
plt.title('Average File Size by Content Type')
plt.xlabel('Content Type')
plt.ylabel('Average Size (MB)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Classification Analysis

Analyzing the unified classifications:

In [ ]:
classification_query = """
WITH classifications AS (
 SELECT 
 json_extract_string(json, '$._source.file_unified_data.classifications_unified.ia_collection[0]') as collection,
 json_extract_string(json, '$._source.file_unified_data.classifications_unified.oclc_holdings[0]') as holdings,
 json_extract_string(json, '$._source.file_unified_data.classifications_unified.content_type[0]') as content_type
 FROM read_parquet('/data/parquet/*.parquet')
)
SELECT 
 collection,
 holdings,
 content_type,
 COUNT(*) as count
FROM classifications
WHERE collection IS NOT NULL
GROUP BY collection, holdings, content_type
HAVING count > 100
ORDER BY count DESC;
"""

df_classifications = conn.execute(classification_query).fetchdf()
display(df_classifications)

## 8. Search Access Types Analysis

Analyzing the distribution of access types:

In [ ]:
access_query = """
WITH access_types AS (
 SELECT 
 UNNEST(json_extract_array(json, '$._source.search_only_fields.search_access_types')) as access_type
 FROM read_parquet('/data/parquet/*.parquet')
)
SELECT 
 access_type,
 COUNT(*) as count,
 ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM access_types
GROUP BY access_type
ORDER BY count DESC;
"""

df_access = conn.execute(access_query).fetchdf()

plt.figure(figsize=(12, 6))
plt.bar(df_access['access_type'], df_access['percentage'])
plt.title('Distribution of Access Types')
plt.xlabel('Access Type')
plt.ylabel('Percentage')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Problems Analysis

Analyzing reported problems in the dataset:

In [ ]:
problems_query = """
WITH problem_records AS (
 SELECT 
 UNNEST(json_extract_array(json, '$._source.file_unified_data.problems')) as problem
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_array(json, '$._source.file_unified_data.problems') IS NOT NULL
)
SELECT 
 problem,
 COUNT(*) as count
FROM problem_records
GROUP BY problem
ORDER BY count DESC;
"""

df_problems = conn.execute(problems_query).fetchdf()
display(df_problems)

## 10. Score Analysis

Analyzing base rank scores:

In [ ]:
score_query = """
WITH score_stats AS (
 SELECT
 json_extract_string(json, '$._source.search_only_fields.search_score_base_rank') as base_rank,
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.search_only_fields.search_score_base_rank') IS NOT NULL
)
SELECT
 content_type,
 COUNT(*) as count,
 ROUND(AVG(CAST(base_rank AS FLOAT)), 2) as avg_rank,
 MIN(CAST(base_rank AS FLOAT)) as min_rank,
 MAX(CAST(base_rank AS FLOAT)) as max_rank
FROM score_stats
GROUP BY content_type
HAVING count > 100
ORDER BY avg_rank DESC;
"""

df_scores = conn.execute(score_query).fetchdf()
display(df_scores)

## 11. Title Length Analysis

Analyzing the distribution of title lengths:

In [ ]:
title_length_query = """
WITH title_stats AS (
 SELECT
 LENGTH(json_extract_string(json, '$._source.file_unified_data.title_best')) as title_length,
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.title_best') IS NOT NULL
)
SELECT
 content_type,
 COUNT(*) as count,
 ROUND(AVG(title_length), 2) as avg_length,
 MIN(title_length) as min_length,
 MAX(title_length) as max_length
FROM title_stats
GROUP BY content_type
HAVING count > 100
ORDER BY avg_length DESC;
"""

df_titles = conn.execute(title_length_query).fetchdf()
display(df_titles)

## 12. Additional Filenames Analysis

Analyzing patterns in additional filenames:

In [ ]:
filename_query = """
WITH filename_patterns AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.original_filename_best') as main_filename,
 UNNEST(json_extract_array(json, '$._source.file_unified_data.original_filename_additional')) as additional_filename
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_array(json, '$._source.file_unified_data.original_filename_additional') IS NOT NULL
)
SELECT
 COUNT(DISTINCT main_filename) as unique_main_files,
 COUNT(DISTINCT additional_filename) as unique_additional_files,
 COUNT(*) as total_relationships
FROM filename_patterns;
"""

df_filenames = conn.execute(filename_query).fetchdf()
display(df_filenames)

## 13. Author Name Variations

Analyzing variations in author names:

In [ ]:
author_variations_query = """
WITH author_vars AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.author_best') as main_author,
 UNNEST(json_extract_array(json, '$._source.file_unified_data.author_additional')) as additional_author
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_array(json, '$._source.file_unified_data.author_additional') IS NOT NULL
)
SELECT
 main_author,
 COUNT(DISTINCT additional_author) as variation_count,
 array_agg(DISTINCT additional_author) as variations
FROM author_vars
GROUP BY main_author
HAVING variation_count > 1
ORDER BY variation_count DESC
LIMIT 10;
"""

df_author_vars = conn.execute(author_variations_query).fetchdf()
display(df_author_vars)

## 14. Publisher Evolution

Analyzing publisher changes over time:

In [ ]:
publisher_evolution_query = """
WITH pub_timeline AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.publisher_best') as publisher,
 TRY_CAST(json_extract_string(json, '$._source.file_unified_data.year_best') AS INTEGER) as year,
 COUNT(*) as publications
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE year BETWEEN 1950 AND 2024
 GROUP BY publisher, year
)
SELECT
 publisher,
 MIN(year) as first_publication,
 MAX(year) as last_publication,
 SUM(publications) as total_publications,
 COUNT(DISTINCT year) as active_years
FROM pub_timeline
WHERE publisher IS NOT NULL
GROUP BY publisher
HAVING total_publications > 1000
ORDER BY total_publications DESC;
"""

df_pub_evolution = conn.execute(publisher_evolution_query).fetchdf()
display(df_pub_evolution)

## 15. Description Analysis

Analyzing book descriptions:

In [ ]:
description_query = """
WITH desc_stats AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.stripped_description_best') as description,
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.stripped_description_best') IS NOT NULL
)
SELECT
 content_type,
 COUNT(*) as total_books,
 ROUND(AVG(LENGTH(description)), 2) as avg_desc_length,
 MIN(LENGTH(description)) as min_length,
 MAX(LENGTH(description)) as max_length
FROM desc_stats
GROUP BY content_type
HAVING total_books > 100
ORDER BY avg_desc_length DESC;
"""

df_descriptions = conn.execute(description_query).fetchdf()
display(df_descriptions)

## 16. Edition Analysis

Analyzing edition variations:

In [ ]:
edition_query = """
WITH edition_data AS (
 SELECT
 json_extract_string(json, '$._source.file_unified_data.edition_varia_best') as main_edition,
 UNNEST(json_extract_array(json, '$._source.file_unified_data.edition_varia_additional')) as additional_edition
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.file_unified_data.edition_varia_best') IS NOT NULL
)
SELECT
 main_edition,
 COUNT(DISTINCT additional_edition) as edition_variations,
 array_agg(DISTINCT additional_edition) as variations
FROM edition_data
GROUP BY main_edition
HAVING edition_variations > 1
ORDER BY edition_variations DESC
LIMIT 10;
"""

df_editions = conn.execute(edition_query).fetchdf()
display(df_editions)

## 17. AACID Analysis

Analyzing AACID patterns:

In [ ]:
aacid_query = """
WITH aacid_patterns AS (
 SELECT
 UNNEST(json_extract_array(json, '$._source.file_unified_data.identifiers_unified.aacid')) as aacid,
 SPLIT_PART(aacid, '__', 2) as source_type,
 SPLIT_PART(aacid, '__', 3) as timestamp
 FROM read_parquet('/data/parquet/*.parquet')
)
SELECT
 source_type,
 COUNT(*) as count,
 MIN(timestamp) as earliest_record,
 MAX(timestamp) as latest_record
FROM aacid_patterns
GROUP BY source_type
ORDER BY count DESC;
"""

df_aacids = conn.execute(aacid_query).fetchdf()
display(df_aacids)

## 18. Search Text Analysis

Analyzing search text patterns:

In [ ]:
search_text_query = """
WITH text_stats AS (
 SELECT
 json_extract_string(json, '$._source.search_only_fields.search_text') as search_text,
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type
 FROM read_parquet('/data/parquet/*.parquet')
 WHERE json_extract_string(json, '$._source.search_only_fields.search_text') IS NOT NULL
)
SELECT
 content_type,
 COUNT(*) as total_records,
 ROUND(AVG(LENGTH(search_text)), 2) as avg_text_length,
 MIN(LENGTH(search_text)) as min_length,
 MAX(LENGTH(search_text)) as max_length
FROM text_stats
GROUP BY content_type
HAVING total_records > 100
ORDER BY avg_text_length DESC;
"""

df_search_text = conn.execute(search_text_query).fetchdf()
display(df_search_text)

## 19. Download Availability Analysis

Analyzing download options:

In [ ]:
download_query = """
SELECT
 json_extract_string(json, '$._source.file_unified_data.content_type_best') as content_type,
 COUNT(*) as total_records,
 SUM(CASE WHEN json_extract(json, '$._source.file_unified_data.has_aa_downloads') = 'true' THEN 1 ELSE 0 END) as aa_downloads,
 SUM(CASE WHEN json_extract(json, '$._source.file_unified_data.has_aa_exclusive_downloads') = 'true' THEN 1 ELSE 0 END) as exclusive_downloads,
 SUM(CASE WHEN json_extract(json, '$._source.file_unified_data.has_torrent_paths') = 'true' THEN 1 ELSE 0 END) as torrent_available,
 SUM(CASE WHEN json_extract(json, '$._source.file_unified_data.has_scidb') = 'true' THEN 1 ELSE 0 END) as scidb_available
FROM read_parquet('/data/parquet/*.parquet')
GROUP BY content_type
HAVING total_records > 100
ORDER BY total_records DESC;
"""

df_downloads = conn.execute(download_query).fetchdf()
display(df_downloads)

## 20. Source Record Chain Analysis

Analyzing source record relationships:

In [ ]:
source_chain_query = """
WITH RECURSIVE source_chain AS (
 SELECT
 json_extract_string(source.value, '$.source_type') as source_type,
 json_extract_string(source.value, '$.source_why') as source_why,
 1 as chain_depth
 FROM read_parquet('/data/parquet/*.parquet'),
 UNNEST(json_extract_array(json, '$._source.source_records')) AS source
 WHERE json_extract_string(source.value, '$.source_why') NOT LIKE '%transitive%'
 
 UNION ALL
 
 SELECT
 json_extract_string(source.value, '$.source_type') as source_type,
 json_extract_string(source.value, '$.source_why') as source_why,
 sc.chain_depth + 1
 FROM read_parquet('/data/parquet/*.parquet'),
 UNNEST(json_extract_array(json, '$._source.source_records')) AS source,
 source_chain sc
 WHERE json_extract_string(source.value, '$.source_why') LIKE '%' || sc.source_type || '%'
 AND chain_depth < 5
)
SELECT
 chain_depth,
 source_type,
 COUNT(*) as occurrences
FROM source_chain
GROUP BY chain_depth, source_type
ORDER BY chain_depth, occurrences DESC;
"""

df_source_chain = conn.execute(source_chain_query).fetchdf()
display(df_source_chain)

# Visualize the chain depth distribution
plt.figure(figsize=(12, 6))
sns.barplot(data=df_source_chain, x='chain_depth', y='occurrences', hue='source_type')
plt.title('Source Record Chain Depth Distribution')
plt.xlabel('Chain Depth')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=0)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()